# Méthode ABC

On compare les résultats obtenus avec les résultats obtenus par la méthode ABC (en 2 dimensions). Cette méthode est normalement utilisé losqu'il n'est pas possible de calculer la vraissemblance, ce qui n'est pas le cas ici.

On suppose les lois priors suivantes (comme pour le MCMC) :
- Pour $\alpha$ : $Exp(2)$
- Pour $Z$ : $\mathcal{N}(0, 10*I_{2*n})$, (avec $n$ le nombre de point)

In [24]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.spatial import distance_matrix

np.random.seed(0)

data = np.loadtxt("data/Florentine_families.csv", delimiter=",") # importe matrice
n = np.shape(data)[0]  # = 15

On réalise donc l'algorithme suivant :

__Entrée__ : $Y^*$: données, N : nombre d'itérations, $\epsilon$: seuil

Sur mille itération : $\newline$
$\ \ $ Tant que $\|\overset{-}{Y}-Y^*\|_1>\epsilon$ : $\newline$
$\ \ \ \ \ $ Tirer $\alpha$ et $Z$ selon le prior $\newline$
$\ \ \ \ \ $ Générer 10 $Y$ selon les $\alpha$ et $Z$ tirés $\newline$
$\ \ \ \ \ $ Poser : $\overset{-}{Y} = \frac{1}{n} \overset{10}{\underset{i=1}\sum} Y_i$

Pour la suite, on prendra :
- $\epsilon = 6$
- $N = 1 000$

In [25]:
N = 10
EPSILON = 45

def generer_Y(alpha:float, Z:np.matrix) -> np.matrix:
    """
    Cette fonction génère Y selon les lois de probabilité données dans l'article
    """
    D = distance_matrix(Z, Z)
    Y = np.zeros(data.shape)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[0]):
            if i!=j:
                proba = np.exp(alpha-D[i][j])/(1+np.exp(alpha-D[i][j]))
                if np.random.random()<proba:
                    Y[i][j] = 1
    return Y

def generer(data: np.matrix) -> tuple[float, float, np.matrix]:
    """
    Cette fonction génère un nouveau alpha et Z selon le prior et les renvoie 
    ainsi que la variable test qui détermine l'acceptation.
    """
    alpha = np.random.exponential(2)
    Z = np.zeros((data.shape[0],2))
    for n in range(Z.shape[0]):
        Z[n] = np.random.multivariate_normal(
            [0 for _ in range(2)], 10 ** (1 / 2) * np.identity(2))
    
    Y_liste = []
    for i in range(10):
        Y_liste.append(generer_Y(alpha, Z))
    Y = np.sum(np.array(Y_liste), axis=0) / 10
    # print(Y)
    test = np.sum(abs(Y-data))
    return test, alpha, Z


alpha_liste = []
Z_liste = []
for _ in range (N):
    test, alpha, Z = generer(data)
    while test>EPSILON :
        test, alpha, Z = generer(data)
    alpha_liste.append(alpha)
    Z_liste.append(Z)

0
1
2
3
4
5
6
7
8
9


On remarque que cet algorithme est très long. (Une minute pour dix valeures, alors qu'il en faudrait des milliers), et alors que la valeur de $\epsilon$ est trop élevée, en effet on obtient les $\alpha$ suivant :

In [27]:
print(alpha_liste)

[0.06797722227501526, 0.22552152808300815, 0.01811442390728763, 0.2245716474677837, 0.09454727224343484, 0.13913100198057288, 0.05619803353420316, 0.1077754554484317, 0.20510135158694115, 0.3294137264808897]
